In [ ]:
import sys
sys.path.append('../input/tabnetdevelop/tabnet-develop (1)/tabnet-develop')

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold
from pytorch_tabnet.tab_model import TabNetRegressor
import numpy as np
import pandas as pd 
import json

import os
import random
import sys
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
from tqdm import tqdm
from sklearn.metrics import log_loss

In [ ]:
data_path = "../input/lish-moa/"
test = pd.read_csv(data_path+'test_features.csv')
test.drop(columns=["sig_id"], inplace=True)
test["cp_type"] = (test["cp_type"]=="trt_cp") + 0
test["cp_dose"] = (test["cp_dose"]=="D1") + 0
X_test = test.values

submission = pd.read_csv(data_path+'sample_submission.csv')

In [ ]:
mask_type = 'entmax'
n_da = 60
n_steps = 1
gamma = 1.0
lambda_sparse = 2.6743669818463933e-05
n_shared = 1

tabnet_params = dict(n_d=n_da, n_a=n_da, n_steps=n_steps, gamma=gamma,
                     lambda_sparse=lambda_sparse, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type=mask_type, n_shared=n_shared,
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=1,
                     )

In [ ]:
# def load_model(model, filepath):
#     """Load TabNet model.
#     Parameters
#     ----------
#     filepath : str
#         Path of the model.
#     """
#     parameterFile = open(f"{filepath}/model_params.json", "r")
#     loaded_params = json.load(parameterFile)
#     saved_state_dict = torch.load(f"{filepath}/network.pt", map_location=torch.device('cuda'))

#     model.__init__(**loaded_params)

#     model._set_network()
#     model.network.load_state_dict(saved_state_dict)
#     model.network.eval()
#     return model

In [ ]:
test_cv_preds = []
model = TabNetRegressor(**tabnet_params)

In [ ]:
dirs = os.listdir(  '../input/tabnetregressor-2-0-3-seed-7-fold' )
dirs.remove('__results__.html') 
dirs.remove('__notebook__.ipynb') 
dirs.remove('custom.css') 
dirs.remove('__output__.json')
dirs.remove('submission.csv')

In [ ]:
dirs

In [ ]:
for x in dirs:
    model.load_model( f"../input/tabnetregressor-2-0-3-seed-7-fold/{x}")
    preds_test = model.predict(X_test)
    test_cv_preds.append(1 / (1 + np.exp(-preds_test)))

### TTA

In [ ]:
mean = np.mean(X_test, axis=0)
std = np.std(X_test, axis=0)

In [ ]:
# gauss = np.random.normal(0, std, 875)
# X_test1 = X_test + gauss
# for x in dirs:
#     model.load_model( f"../input/tabnetregressor-2-0-3-seed-7-fold/{x}")
#     preds_test = model.predict(X_test1)
#     test_cv_preds.append(1 / (1 + np.exp(-preds_test)))

In [ ]:
# gauss = np.random.normal(0, std, 875)
# X_test2 = X_test + gauss
# for x in dirs:
#     model.load_model( f"../input/tabnetregressor-2-0-3-seed-7-fold/{x}")
#     preds_test = model.predict(X_test2)
#     test_cv_preds.append(1 / (1 + np.exp(-preds_test)))

In [ ]:
# gauss = np.random.normal(0, std, 875)
# X_test3 = X_test + gauss
# for x in dirs:
#     model.load_model( f"../input/tabnetregressor-2-0-3-seed-7-fold/{x}")
#     preds_test = model.predict(X_test3)
#     test_cv_preds.append(1 / (1 + np.exp(-preds_test)))

In [ ]:
test_preds_all = np.stack(test_cv_preds)
all_feat = [col for col in submission.columns if col not in ["sig_id"]]
submission[all_feat] = test_preds_all.mean(axis=0)
# set control to 0
submission.loc[test['cp_type']==0, submission.columns[1:]] = 0
submission.to_csv('submission.csv', index=None)

In [ ]:
submission